In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '../../')
from src.encode_data import *
from src.midi_data import *
from src.data_sources import process_all, arr2csv
from src.midi_transform import *
from src.fastai_data import *

In [3]:
import traceback
import time

## Standardize and reformat raw midi files before encoding to text
- Transform key to C major
- Remove unused instruments
- Combine multiple tracks with the same instrument into a single part
- Melody, Piano, String

### Load midi data

In [4]:
version = 'v17'
data_path = Path('data/midi')
version_path = data_path/version

In [5]:
import pandas as pd

In [6]:
sf_path = f'sf{SAMPLE_FREQ}'
out_dir = Path(f'{sf_path}/midi_encode')

In [7]:
out_csv = version_path/out_dir/f'{out_dir.name}.csv'
out_csv.parent.mkdir(parents=True, exist_ok=True)
out_csv

PosixPath('data/midi/v17/sf4/midi_encode/midi_encode.csv')

### Encoding midi to numpy

In [8]:
df = pd.read_csv(out_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,2,5,6,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,song_url,md5,mxl,genres,ht_offset,ht_key,section,source,ht_bpm,artist,ht_mode,numpy,midi_title,title,midi,parts,ht_time_signature
0,https://www.hooktheory.com/theorytab/view/wayn...,bf1f29e5ff84e3e93e37fb873bfb590e,NaN,NaN,0.0,C,chorus,hooktheory,128.0,wayne-sharpe,1.0,sf4/midi_encode/hooktheory/pianoroll/w/wayne-s...,yu-gi-oh3,yu-gi-oh-theme-song,midi_sources/hooktheory/pianoroll/w/wayne-shar...,"intro,chorus",4.0
1,https://www.hooktheory.com/theorytab/view/wayn...,055f80ad67f64edb14a85ca8fbfe8c29,NaN,NaN,0.0,C,intro,hooktheory,85.0,wayne-sharpe,1.0,sf4/midi_encode/hooktheory/pianoroll/w/wayne-s...,yu-gi-oh,yu-gi-oh-theme-song,midi_sources/hooktheory/pianoroll/w/wayne-shar...,"intro,chorus",3.0
2,https://www.hooktheory.com/theorytab/view/what...,197f96f5d181f6ce1e2c5ab04ac1ff87,NaN,Jazz,-5.0,D,chorus,hooktheory,96.0,what-a-day,6.0,sf4/midi_encode/hooktheory/pianoroll/w/what-a-...,kiefer,kiefer,midi_sources/hooktheory/pianoroll/w/what-a-day...,chorus,4.0
3,https://www.hooktheory.com/theorytab/view/whit...,9e7ce13a35f1314423a9a6d5a5287a4a,NaN,"J-Pop,Pop",-5.0,D,pre-chorus,hooktheory,152.0,whiteflame,6.0,sf4/midi_encode/hooktheory/pianoroll/w/whitefl...,senbonzakura - pre-Pre-Chorus,senbonzakura,midi_sources/hooktheory/pianoroll/w/whiteflame...,"verse,pre-chorus,chorus",4.0
4,https://www.hooktheory.com/theorytab/view/whit...,d5aaf79d0989222f1362f9f46c540a27,NaN,"J-Pop,Pop",-5.0,D,verse,hooktheory,152.0,whiteflame,6.0,sf4/midi_encode/hooktheory/pianoroll/w/whitefl...,Senbonzakura,senbonzakura,midi_sources/hooktheory/pianoroll/w/whiteflame...,"verse,pre-chorus,chorus",4.0


In [9]:
all_records = df.to_dict(orient='records'); len(all_records)

196531

In [14]:
import pdb

In [10]:
def process_metadata(metadata):
    result = metadata.copy()
    
    if not isinstance(metadata.get('numpy'), str): return None
    
    input_path = version_path/'old'/metadata['numpy']
    output_path = version_path/metadata['numpy']
    
    if not input_path.exists(): return
    if output_path.exists(): return
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    np_array = np.load(input_path)
    pdb.set_trace()
    if len(np_array.shape) == 1:
        if np_array[0][-1, 0] == VALTSEP:
            np_array[0] = np_array[0][:-1]
        np_array[0] =
    elif np_array[-1, 0] == VALTSEP:
        np_array = np_array[:-1]
        np.save(output_path, np_array)
    else:
        print('Last timestep is not encoded correctly:', input_path)

In [11]:
# # sanity check
import random
for r in random.sample(all_records, 10):
    process_metadata(r)

In [12]:
def timeout_func(data, seconds):
    print("Timeout:", seconds, data.get('midi'))

In [13]:
timeout = 500
processed = process_all(process_metadata, all_records, timeout=timeout, timeout_func=timeout_func)

In [ ]:
arr2csv(processed, out_csv); len(processed)

In [ ]:
df = pd.read_csv(out_csv); df.head()

In [19]:
len([s for s in df.midi.values if isinstance(s, str)])

196531

In [20]:
len([s for s in df.numpy.values if isinstance(s, str)])

164774

In [38]:
len([s for s in df.numpy.values if isinstance(s, str)])

112169

In [39]:
Counter(df[df.midi.notnull()].source.values)

Counter({'hooktheory': 19882,
         'freemidi': 5168,
         'midiworld': 4108,
         'ecomp': 2508,
         'cprato': 312,
         'classical_piano': 329,
         'classical_archives': 14642,
         'musescore': 10933,
         'wikifonia': 6346,
         'lmd': 13565,
         'reddit': 98674,
         'hooktheory_c': 20076})

In [22]:
Counter(df[df.numpy.notnull()].source.values)

Counter({'hooktheory': 19404,
         'freemidi': 3974,
         'midiworld': 3460,
         'ecomp': 2508,
         'cprato': 308,
         'classical_piano': 329,
         'classical_archives': 14627,
         'musescore': 10891,
         'wikifonia': 6195,
         'lmd': 9175,
         'reddit': 74341,
         'hooktheory_c': 19562})